In [1]:
import pandas as pd
import numpy as np
import pickle as pickle
import os

In [2]:
# Prepare Project Implicit data for diffusion modeling

#Settings
# Path to data
PATH = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/data/rts/'

# Path to outcomes (age and session ID)
PATH_OUTCOME = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/data/outcomes/'

# Where to save prepared files
PATH_TO_SAVE = '/home/mischa/Documents/bayesflow/prj_real_life_ddm/data/prepared_data/'

In [4]:
def read_iat_rt_file(file_to_read, black_not_african):
    """
    Reads and preprocesses a single IAT file. "black_not_african" relates to the stimulus type of the word stimuli.
    """
    
    if black_not_african:
        pairs_to_retain = ['Black people/Bad,White people/Good', 
                            'White people/Bad,Black people/Good', 
                            'White people/Good,Black people/Bad', 
                            'Black people/Good,White people/Bad'
                           "Black American/Bad,White American/Good",
                           "Black American/Good,White American/Bad",
                           "White American/Bad,Black American/Good",
                           "White American/Good,Black American/Bad", 
                          "White/Bad,Black/Good", 
                           "White/Good,Black/Bad",
                          "Black/Bad,White/Good",
                          "Black/Good,White/Bad"]
    else:
        pairs_to_retain =['African Americans/Bad,European Americans/Good', 
                            'European Americans/Bad,African Americans/Good', 
                            'European Americans/Good,African Americans/Bad', 
                            'African Americans/Good,European Americans/Bad',
                         'African American/Bad,White American/Good',
                         'African American/Good,White American/Bad',
                         'White American/Bad,African American/Good',
                         'White American/Good,African American/Bad',
                        'African American/Bad,European American/Good',
                         'African American/Good,European American/Bad',
                         'European American/Bad,African American/Good',
                         'European American/Good,African American/Bad']                                             
    
    # We only need certain column from the data, so we can indicate pandas to only read
    # a subset of them and save memory. The problem is, when the header is read, 
    # there are whitespaces in the data, so we shall assume
    # that all relevant columns have the same order in all datafiles.
    cols_to_use_str = ['block_number', 'block_pairing_definition', 'trial_name','trial_latency', 'trial_error', 
                       'session_id']
    cols_to_use_idx = [0, 3, 10, 8, 11, 12]
    valid_values_error = [0, 1, '1', '0'] # Use for some datasets containing invalid values
    
    df = pd.read_csv(PATH + file_to_read, delimiter='\t', usecols=cols_to_use_idx, low_memory=False)

    # Fix data handling 
    # There are whitespaces in the columns, to we need to get rid of them
    df.columns = [c.strip() for c in df.columns]
    assert all([c1 == c2 for c1, c2 in zip(df.columns, cols_to_use_str)]), 'Expected column order does not match!'
 
    # Retain only relevant pairs
    df = df[df['block_pairing_definition'].isin(pairs_to_retain)]

    print('Initial shape:', len(df.groupby('session_id')))

    # Remove ids with no age        
    df['session_id'] = df['session_id'].astype("string")
    df = df[df['session_id'].isin(outcomes['session_id'])]
    print('After removing no age IDs:', len(df.groupby('session_id')))
   
    # Remove invalid ids
    df = df[df['trial_error'].isin(valid_values_error)]
    print('After removing invalid IDs:', len(df.groupby('session_id')))
    
    # Ensure correct types
    df = df.astype({"block_number": int, 
                    "block_pairing_definition": str,
                    "trial_latency": int,
                    "trial_error": int})
      
    # Remove sessions with 0 latency
    zero_latency_id = df[df['trial_latency']==0]['session_id'].to_list()
    df = df[~df['session_id'].isin(zero_latency_id)]
    print('After excluding zero latency IDs:', len(df.groupby('session_id')))
    
    # Remove sessions with not exactly 120 trials
    not120trials_id = df.groupby('session_id').size()
    not120trials_id = not120trials_id[not120trials_id != 120].index.to_list()

    df = df[~df['session_id'].isin(not120trials_id)]
    print('After excluding != 120 trials IDs:', len(df.groupby('session_id')))
    
    # Remove sessions with >50% errors
    fiftypercent_errors_id = df.groupby('session_id').mean(numeric_only=True)
    fiftypercent_errors_id = fiftypercent_errors_id[fiftypercent_errors_id['trial_error'] > 0.5].index.to_list()
    df = df[~df['session_id'].isin(fiftypercent_errors_id)]
    print('After excluding more than 50% error IDs:', len(df.groupby('session_id')))

    # Remove sessions with 0 errors
    zero_errors_id = df.groupby('session_id').sum(numeric_only=True)
    zero_errors_id = zero_errors_id[zero_errors_id['trial_error'] == 0].index.to_list()
    df = df[~df['session_id'].isin(zero_errors_id)]
    print('After excluding zero error IDs:', len(df.groupby('session_id')))

    # Convert RTs in ms
    df.loc[:, 'trial_latency'] = df['trial_latency'] / 1000
             
    # Code compatible vs incompatible
    # Changes for different types of stimuli! (black/white vs african american/european american)
    if black_not_african:
        df['condition_dummy'] =        np.where((df['block_pairing_definition'] == "Black people/Bad,White people/Good") |
                                                 (df['block_pairing_definition'] == 'White people/Good,Black people/Bad') |
                                                 (df['block_pairing_definition'] == 'Black American/Bad,White American/Good') |
                                                 (df['block_pairing_definition'] == 'Black/Bad,White/Good') |
                                                 (df['block_pairing_definition'] == 'White/Good,Black/Bad') |
                                                 (df['block_pairing_definition'] == 'White American/Good,Black American/Bad'), 0, 1)
    else:
        df['condition_dummy'] =        np.where((df['block_pairing_definition'] == "African Americans/Bad,European Americans/Good") |
                                                 (df['block_pairing_definition'] == 'European Americans/Good,African Americans/Bad') |
                                                 (df['block_pairing_definition'] == "African American/Bad,European American/Good") |
                                                 (df['block_pairing_definition'] == "African American/Bad,White American/Good") |
                                                 (df['block_pairing_definition'] == 'European American/Good,African American/Bad')|
                                                 (df['block_pairing_definition'] == 'White American/Good,African American/Bad'), 0, 1)
        
    
    # Code word vs. picture, picture == 1, word == 0
    df['stimulus'] = np.where((df['trial_name'].str.contains('jpg|gif')), 1, 0)

    
    # Sort dataframe according to session id
    df = df.sort_values('session_id')
    
    # Convert rt dataset to dictionary (session_id - np.array of rt/errors/compatible/stimulustype)
    df_rt_dict = df.groupby('session_id', group_keys=False)['trial_latency'].apply(np.array).to_dict()
    df_err_dict = df.groupby('session_id', group_keys=False)['trial_error'].apply(np.array).to_dict()
    df_cond_dict = df.groupby('session_id', group_keys=False)['condition_dummy'].apply(np.array).to_dict()
    df_stimulus_dict = df.groupby('session_id', group_keys=False)['stimulus'].apply(np.array).to_dict()

    
    rts_data = {}
    for k1, k2, k3, k4 in zip(df_rt_dict, df_err_dict, df_cond_dict,df_stimulus_dict):
        assert k1 == k2 == k3 == k4, 'Key mismatch within the same dataset, something went wrong!'
        rts_data[k1] = np.c_[df_rt_dict[k1], df_err_dict[k1],df_cond_dict[k1], df_stimulus_dict[k1]]
    return rts_data


def prepare_data(dataset_name, black_not_african):
    """
    Preprocesses a single IAT file and combines with outcomes. 
    Remove cases with more than 20 trials < 200 ms or > 10 seconds
    """    
    # Prepare a list for current chunk data/outcomes
    # Since we don't know how many will survive the next step
    # we initialize lists which we late convert to np.arrays

    X_arr = []
    y_arr = []
    
    # Read in file
    data_dict = read_iat_rt_file(dataset_name, black_not_african)
    print('Successfully read datafile', dataset_name, '.')
    print('Converting to X and y arrays...')

    # Loop through sessions and get those RT arrays for which we have outcomes
    for key in data_dict.keys():
        outcome = outcomes_dict.get(key)
        
        # Add only if outcome for the dataset is present
        if outcome is not None:
            
            # Add only if 120 trials and 2 outcomes (age and ID) present
            if (data_dict[key].shape == (120, 4)) and outcomes_dict[key].shape == (2,):
                X_arr.append(data_dict[key])
                y_arr.append(outcomes_dict[key])
            
    # X becomes a 3D array (N_datasets x 120 x 4)
    # y becomes a 2D array (N_datasets x 2)
    X_test = np.stack(X_arr)
    y_test = np.stack(y_arr)
        
    # Get X_test into the correct format:
    rts = np.where(X_test[:, :, 1], -X_test[:, :, 0], X_test[:, :, 0])
    cond = X_test[:, :, 2]
    stimulus = X_test[:, :, 3]

    X_test = np.stack((rts, cond, stimulus), axis=2)
    
    # Set trials with latency <200 ms or >10 seconds to 0
    X_test[:,:,0][np.abs(X_test[:,:,0]) < .2] = 0
    X_test[:,:,0][np.abs(X_test[:,:,0]) > 10] = 0

    # Exclusion criterion (>=20 trials missing)
    idx_not_careless = (np.abs(X_test[:, :, 0]) == 0).sum(axis=1) <= 20
    X_test = X_test[idx_not_careless, :, :]
    print('IDs with less than 20 trials <200 ms or >10 seconds:', X_test.shape)
    
    # Keep only corresponding y
    y_test = y_test[idx_not_careless, :]
    
    # Exclusion criterion (60 trials for each condition, 60 trials for both picture and word stimuli)
    idx_experiment_not_broken = (
        ((np.abs(X_test[:, :, 1]) == 0).sum(axis=1) == 60) &
        ((np.abs(X_test[:, :, 1]) == 1).sum(axis=1) == 60) &
        ((np.abs(X_test[:, :, 2]) == 0).sum(axis=1) == 60) &
        ((np.abs(X_test[:, :, 2]) == 1).sum(axis=1) == 60)
        )
    
    X_test = X_test[idx_experiment_not_broken, :, :]
    print('IDs with 60 trials for each condition, 60 trials for both picture and word stimuli:', X_test.shape)

    # Keep only corresponding y
    y_test = y_test[idx_experiment_not_broken, :]
    y_test = np.int64(y_test)
       
    # Create dummy coding for missing data
    missings = np.zeros((X_test.shape[0],X_test.shape[1]))
    missings[X_test[:,:,0]==0] = 1  

    # Concatenate rt, missings, conditions (note that the axis numberings have been changed above)
    X_test = np.stack((X_test[:, :, 0], missings,
                       X_test[:, :, 1], X_test[:, :, 2]), axis=2).astype(np.float32)

    print('Final shape of RTs chunk (after once more excluding cases w/ missing age or trials): ', X_test.shape)
    print('Final shape of outcomes chunk: ', X_test.shape)
    
    return X_test, y_test

def process_rt_data(raw_data):
    """
    Create summary statistics for RTs and accuracies
    
    """   
    array_for_processed_data = np.zeros(shape=(raw_data.shape[0], 20))
    array_for_processed_data[:] = np.nan
    
    #for loop over people
    for person in range(raw_data.shape[0]):
        #congruent trials
        congruent_rt_correct = np.mean(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,2]==0),
                                                   0], axis=0)

        congruent_rt_correct_sd = np.std(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,2]==0),
                                                   0], axis=0)
            
        congruent_rt_error = np.abs(np.mean(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,2]==0),
                                                        0], axis=0))
        
        congruent_rt_error_sd = np.abs(np.std(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,2]==0),
                                                        0], axis=0))
        

        congruent_accuracy = (np.sum(raw_data[person,raw_data[person,:,2]==0,0]>0, axis=0) /
                              np.sum(raw_data[person,raw_data[person,:,2]==0,0]!=0, axis=0))

        # incongruent trials

        incongruent_rt_correct = np.mean(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,2]==1),
                                                   0], axis=0)
        
        incongruent_rt_correct_sd = np.std(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,2]==1),
                                                   0], axis=0)

        incongruent_rt_error = np.abs(np.mean(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,2]==1),
                                                        0], axis=0))
        
        incongruent_rt_error_sd = np.abs(np.std(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,2]==1),
                                                        0], axis=0))

        incongruent_accuracy = (np.sum(raw_data[person,raw_data[person,:,2]==1,0] > 0, axis=0) /
                              np.sum(raw_data[person,raw_data[person,:,2]==1,0] != 0, axis=0))

        # word trials

        word_rt_correct = np.mean(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,3]==0),
                                                   0], axis=0)
                                         
        word_rt_correct_sd = np.std(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,3]==0),
                                                   0], axis=0)

        word_rt_error = np.abs(np.mean(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,3]==0),
                                                        0], axis=0))
        word_rt_error_sd = np.abs(np.std(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,3]==0),
                                                        0], axis=0))
                                                                                  
        word_accuracy = (np.sum(raw_data[person,raw_data[person,:,3]==0,0] > 0, axis=0) /
                              np.sum(raw_data[person,raw_data[person,:,3]==0,0] != 0, axis=0))

        # picture trials

        picture_rt_correct = np.mean(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,3]==1),
                                                   0], axis=0)
                                         
        picture_rt_correct_sd = np.std(raw_data[person,
                                                   (raw_data[person,:,0]>0)&(raw_data[person,:,3]==1),
                                                   0], axis=0)

        picture_rt_error = np.abs(np.mean(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,3]==1),
                                                        0], axis=0))
                                         
        picture_rt_error_sd = np.abs(np.std(raw_data[person,
                                                        (raw_data[person,:,0]<0)&(raw_data[person,:,3]==1),
                                                        0], axis=0))

        picture_accuracy = (np.sum(raw_data[person,raw_data[person,:,3]==1,0] > 0, axis=0) /
                              np.sum(raw_data[person,raw_data[person,:,3]==1,0] != 0, axis=0))

        array_for_processed_data[person,:] = (congruent_rt_correct, congruent_rt_correct_sd,
                                              congruent_rt_error, congruent_rt_error_sd, congruent_accuracy,
                                              incongruent_rt_correct, incongruent_rt_correct_sd,
                                              incongruent_rt_error, incongruent_rt_error_sd, incongruent_accuracy,
                                              word_rt_correct, word_rt_correct_sd,
                                              word_rt_error, word_rt_error_sd, word_accuracy,
                                              picture_rt_correct, picture_rt_correct_sd,
                                              picture_rt_error, picture_rt_error_sd, picture_accuracy)
    return array_for_processed_data

In [5]:
# Convert outcomes to a dictionary for instant lookup; use only cases containing age and session id
#outcomes_id2_dict = outcomes.groupby('session_id')['id2'].apply(int).to_dict()
#outcomes_age_dict = outcomes.groupby('session_id')['age'].apply(int).to_dict()
#outcomes_dict = {}

#for k1, k2 in zip(outcomes_id2_dict, outcomes_age_dict):
#    assert k1 == k2 
#    outcomes_dict[k1] = np.array([outcomes_id2_dict[k1], 
#                                  outcomes_age_dict[k2]])
    
# Note, that outcomes dict will be a global dictionary

#pickle.dump(outcomes_dict,
#                    open(PATH_OUTCOME +'pickle_outcomes.p' ,"wb"))


# Read in outcomes, drop missing, sort
outcomes = pd.read_csv(PATH_OUTCOME + 'outcomes20032023_id_age_only.csv', header=0, na_values='NA')
outcomes = outcomes.dropna().drop_duplicates()
outcomes = outcomes.sort_values('session_id')
outcomes['session_id'] = outcomes['session_id'].astype("string")

# Open pre-compiled outcomes dictionary
outcomes_dict = pickle.load(open(PATH_OUTCOME +'pickle_outcomes.p' ,"rb"))

In [7]:
datasets = sorted(os.listdir(PATH))

#For each chunk
for black_not_african in (False, True):
    
    for dataset_name in datasets:
    
        # 1. Load and prepare chunk
        try:
            X_test, y_test = prepare_data(dataset_name, black_not_african)
        except:
            continue

        # 2. Prepare RT summaries
        summary_stats = process_rt_data(X_test)
        
        # 3. Save pickle
        dict_to_store = {'data_array': X_test, "outcome_array": y_test, 'rt_summaries': summary_stats}
        
        pickle.dump(dict_to_store,
                    open(
                        PATH_TO_SAVE+ "prepared_"+str(black_not_african)+ str(dataset_name.replace('.txt',
                                                                                                   '.p') ),"wb"))
        print(str(dataset_name)+ " done")

Initial shape: 37709
After removing no age IDs: 31845
After removing invalid IDs: 31845
After excluding zero latency IDs: 31826
After excluding != 120 trials IDs: 31496
After excluding more than 50% error IDs: 31447
After excluding zero error IDs: 30678
Successfully read datafile 2003iat.txt .
Converting to X and y arrays...
IDs with less than 20 trials <200 ms or >10 seconds: (30649, 120, 3)
IDs with 60 trials for each condition, 60 trials for both picture and word stimuli: (30649, 120, 3)
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (30649, 120, 4)
Final shape of outcomes chunk:  (30649, 120, 4)


/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


2003iat.txt done
Initial shape: 25524
After removing no age IDs: 4067
After removing invalid IDs: 4067
After excluding zero latency IDs: 4061
After excluding != 120 trials IDs: 3986
After excluding more than 50% error IDs: 3982
After excluding zero error IDs: 3868
Successfully read datafile 2004iat.txt .
Converting to X and y arrays...
IDs with less than 20 trials <200 ms or >10 seconds: (3840, 120, 3)
IDs with 60 trials for each condition, 60 trials for both picture and word stimuli: (3840, 120, 3)
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (3840, 120, 4)
Final shape of outcomes chunk:  (3840, 120, 4)


/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


2004iat.txt done
Initial shape: 25519
After removing no age IDs: 20304
After removing invalid IDs: 20304
After excluding zero latency IDs: 20293
After excluding != 120 trials IDs: 20103
After excluding more than 50% error IDs: 20091
After excluding zero error IDs: 19389
Successfully read datafile 2005iat.txt .
Converting to X and y arrays...
IDs with less than 20 trials <200 ms or >10 seconds: (19281, 120, 3)
IDs with 60 trials for each condition, 60 trials for both picture and word stimuli: (19281, 120, 3)
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (19281, 120, 4)
Final shape of outcomes chunk:  (19281, 120, 4)


...

20232iat3.txt done
Initial shape: 26013
After removing no age IDs: 25998
After removing invalid IDs: 25998
After excluding zero latency IDs: 25990
After excluding != 120 trials IDs: 24809
After excluding more than 50% error IDs: 24750
After excluding zero error IDs: 22540
Successfully read datafile 20232iat4.txt .
Converting to X and y arrays...
IDs with less than 20 trials <200 ms or >10 seconds: (22325, 120, 3)
IDs with 60 trials for each condition, 60 trials for both picture and word stimuli: (22325, 120, 3)
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (22325, 120, 4)
Final shape of outcomes chunk:  (22325, 120, 4)


/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


20232iat4.txt done
Initial shape: 14652
After removing no age IDs: 14637
After removing invalid IDs: 14637
After excluding zero latency IDs: 14632
After excluding != 120 trials IDs: 13853
After excluding more than 50% error IDs: 13815
After excluding zero error IDs: 12649
Successfully read datafile 20232iat5.txt .
Converting to X and y arrays...
IDs with less than 20 trials <200 ms or >10 seconds: (12541, 120, 3)
IDs with 60 trials for each condition, 60 trials for both picture and word stimuli: (12541, 120, 3)
Final shape of RTs chunk (after once more excluding cases w/ missing age or trials):  (12541, 120, 4)
Final shape of outcomes chunk:  (12541, 120, 4)


/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/mischa/miniconda3/envs/bf/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


20232iat5.txt done
